In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.profiler import profile, record_function, ProfilerActivity
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tqdm import tqdm
import random
import json
import seaborn as sns
import time

## Data Preparation & Preprocessing

In [3]:
class DataPreparation():
    def __init__(self, dataset_filename, num_steps=None, input_size=None, configs_filename='configs', seed=42, period='1S'):
        self.num_steps = num_steps
        self.input_size = input_size
        self.set_seed(seed)
        self.load_configs(configs_filename)
        self.load_data(dataset_filename, period)
        
    def set_seed(self, seed):
        np.random.seed(seed)  # Set seed for numpy
        random.seed(seed)  # Set seed for random
        
        if torch.cuda.is_available():
            torch.manual_seed(seed)  # Set seed for PyTorch CPU
        
        torch.cuda.manual_seed(seed)  # Set seed for PyTorch GPU
        torch.cuda.manual_seed_all(seed)  # Set seed for all GPUs
        torch.backends.cudnn.deterministic = True  # Ensure deterministic behavior for CUDA
        torch.backends.cudnn.benchmark = False  # Disable the auto-tuner for GPUs
    
    def load_configs(self, configs_filename):
        # Load hyperparameters
        with open(f'configs/{configs_filename}.json', 'r') as file:
            self.hyperparams = json.load(file)
            
    def load_data(self, dataset_filename, period='1S'):
        self.data = pd.read_csv('data/' + dataset_filename + '.csv')
        self.data = extract_high_frequency_trading(self.data, period)
        # self.data = self.data.resample(period).mean()  # Resample data based on the period
        
        # Extract attributes
        if self.num_steps is None:
            self.num_steps = self.hyperparams['num_steps']  # Extract number of steps
        
        if self.input_size is None:
            self.input_size = self.hyperparams['input_size']  # Extract input size
        
        self.val_split = self.hyperparams['val_split']  # Extract validation split
        self.test_split = self.hyperparams['test_split']  # Extract test split
        self.batch_size = self.hyperparams['batch_size'] # Extract batch size
        self.num_workers = self.hyperparams['num_workers'] # Extract number of workers (for GPU)
        
        # Normalize data
        self.data = self.normalize_data(self.data)

        # Assert no NaN values
        assert(self.data.isna().sum().sum() == 0)

        # Create sequences (with sliding windows)
        X_train, y_train, X_val, y_val, X_test, y_test = self.create_sequences(self.data, self.input_size, self.num_steps, self.val_split, self.test_split)
        
        # Convert to PyTorch tensors
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
        X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
        y_val_tensor = torch.tensor(y_val, dtype=torch.float32)
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
        y_test_tensor = torch.tensor(y_test, dtype=torch.float32)
        
        assert(not torch.isnan(X_train_tensor).any())
        assert(not torch.isnan(y_train_tensor).any())
        assert(not torch.isnan(X_val_tensor).any())
        assert(not torch.isnan(y_val_tensor).any())
        assert(not torch.isnan(X_test_tensor).any())
        assert(not torch.isnan(y_test_tensor).any())
        
        # Create DataLoader instances
        self.train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=self.batch_size, shuffle=True, num_workers=self.num_workers, pin_memory=True)
        self.val_loader = DataLoader(TensorDataset(X_val_tensor, y_val_tensor), batch_size=self.batch_size, shuffle=False, num_workers=self.num_workers, pin_memory=True)
        self.test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=self.batch_size, shuffle=False, num_workers=self.num_workers, pin_memory=True)

    def normalize_data(self, data):
        # Apply MinMaxScaler to normalize data
        scaler = MinMaxScaler()
        normalized_data = scaler.fit_transform(data)
        return pd.DataFrame(normalized_data, index=data.index, columns=data.columns)
    
    def create_sequences(self, data, input_size, num_steps, val_split=0.1, test_split=0.1):
        # Convert data to numpy array
        data = np.array(data)
    
        # Check if data has any NaNs
        if np.isnan(data).any():
            raise ValueError("Data contains NaN values. Please clean the data before proceeding.")
    
        # Normalize data
        # Replace zeros in the last price with a small value to avoid division by zero
        data = np.where(data[:, -1] == 0, 1e-6, data[:, -1])  # Small constant to avoid division by zero
        data = [data[i * input_size: (i + 1) * input_size] 
               for i in range(len(data) // input_size)]
        
        # Normalize the data
        data = [data[0] / data[0][0] - 1.0] + [
                    curr / data[i][-1] - 1.0 for i, curr in enumerate(data[1:])]
        
        # Check for remaining NaNs after normalization
        if np.isnan(data).any():
            raise ValueError("Data contains NaN values after normalization. Please check the normalization process.")
        
        # Split into groups of `num_steps`
        X = np.array([data[i: i + num_steps] for i in range(len(data) - num_steps)])
        y = np.array([data[i + num_steps] for i in range(len(data) - num_steps)])
    
        # Drop all rows with NaN values
        mask = ~np.isnan(X).any(axis=(1, 2)) & ~np.isnan(y).any(axis=1)
        X = X[mask]
        y = y[mask]
        
        # Reshape X to have shape (N, num_steps, input_size)
        X = X.reshape(-1, num_steps, input_size)
        y = y.reshape(-1, input_size)  # Reshape y to match the output shape
        
        # Split into train, validation, and test sets
        total_len = len(X)
        test_start = int(total_len * (1 - test_split))
        val_start = int(total_len * (1 - test_split - val_split))
        
        X_train, X_val, X_test = X[:val_start], X[val_start:test_start], X[test_start:]
        y_train, y_val, y_test = y[:val_start], y[val_start:test_start], y[test_start:]
        
        return X_train, y_train, X_val, y_val, X_test, y_test


def extract_high_frequency_trading(data, period='1S'):
    # Convert datetime column to pandas datetime type
    data['datetime'] = pd.to_datetime(data['datetime'])
    
    # # Add additional diffs info
    # data_pre = data.copy().shift(1)
    # data_pre['price_diff'] = data['last_price'] - data_pre['last_price']
    # data_pre['oi_diff'] = data['open_interest'] - data_pre['open_interest']
    # data_pre['vol_diff'] = data['volume'] - data_pre['volume']
    
    # data_pre["pc"] = np.where(
    #     data["last_price"] <= data_pre["bid_price1"], -1,
    #     np.where(data["last_price"] >= data_pre["ask_price1"], 1,
    #              np.sign(data_pre["price_diff"])))
    # pc_g = data_pre["pc"] > 0
    
    
    # condition_buy = (data["last_price"] >= data_pre["ask_price1"]) | (data["last_price"] >= data["ask_price1"])
    # condition_sell = (data['last_price'] <= data_pre['bid_price1']) | (data['last_price'] <= data['bid_price1'])
    # direction values = np.where(condition_buy,np.where(condition_sell,"主动卖！", " 其他，"))data_pre['direction'] = pd.Series (direction _values, index=data.index)
    
    # Identify high-frequency trading periods
    # Assuming a significant increase in volume and consistent bid/ask prices indicates high-frequency trading
    # Adjust the thresholds as necessary for your dataset
    high_freq_trading = data[
        (data['volume'] > 0) & 
        (data['open_interest'] > 0) & 
        (data['last_price'].notna()) & 
        (data['highest'].notna()) & 
        (data['volume'].notna()) & 
        (data['amount'].notna()) & 
        (data['bid_price1'].notna()) & 
        (data['ask_price1'].notna())
    ]
    
    # # Calculate the difference between consecutive rows for open_interest and volume
    # high_freq_trading['open_interest_diff'] = high_freq_trading['open_interest'].diff()
    # high_freq_trading['volume_diff'] = high_freq_trading['volume'].diff()
    
    # # Find the row with the maximum increase in open_interest
    # max_open_interest_idx = high_freq_trading['open_interest_diff'].idxmax()
    # max_open_interest_segment = high_freq_trading.loc[max_open_interest_idx - 1 : max_open_interest_idx + 1]
    
    # # Find the row with the maximum increase in volume
    # max_volume_idx = high_freq_trading['volume_diff'].idxmax()
    # max_volume_segment = high_freq_trading.loc[max_volume_idx - 1 : max_volume_idx + 1]
    
    # Additional filter to ensure high-frequency (optional)
    # You can adjust the frequency threshold as needed
    high_freq_trading = high_freq_trading.set_index('datetime')
    high_freq_trading = high_freq_trading.resample(period).ffill().dropna()  # Resample to `period` intervals and forward fill

    # Ensure No NaN values remaining
    assert(high_freq_trading.isna().sum().sum() == 0)
    
    # Return the filtered dataset
    return high_freq_trading

In [4]:
data = pd.read_csv('data/INE.sc2010.csv')
data = extract_high_frequency_trading(data, period='1S')
data

,last_price,highest,lowest,bid_price1,bid_volume1,bid_price2,bid_volume2,bid_price3,bid_volume3,bid_price4,...,ask_volume2,ask_price3,ask_volume3,ask_price4,ask_volume4,ask_price5,ask_volume5,volume,amount,open_interest
datetime,,,,,,,,,,,,,,,,,,,,,
2019-10-30 09:10:46,447.2,447.2,447.2,430.1,1.0,430.0,1.0,424.4,1.0,422.4,...,1.0,447.3,1.0,464.6,1.0,503.9,1.0,1.0,447200.0,1.0
2019-10-30 09:10:47,447.2,447.2,447.2,430.1,1.0,430.0,1.0,424.4,1.0,422.4,...,1.0,447.3,1.0,464.6,1.0,503.9,1.0,1.0,447200.0,1.0
2019-10-30 09:10:48,447.2,447.2,447.2,430.1,1.0,430.0,1.0,424.4,1.0,422.4,...,1.0,447.3,1.0,464.6,1.0,503.9,1.0,1.0,447200.0,1.0
2019-10-30 09:10:49,447.2,447.2,447.2,430.1,1.0,430.0,1.0,424.4,1.0,422.4,...,1.0,447.3,1.0,464.6,1.0,503.9,1.0,1.0,447200.0,1.0
2019-10-30 09:10:51,447.2,447.2,447.2,431.1,1.0,431.0,1.0,424.4,1.0,422.4,...,1.0,447.3,1.0,464.6,1.0,503.9,1.0,1.0,447200.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-09-23 14:22:28,250.0,260.0,234.5,248.2,5.0,242.2,6.0,242.0,10.0,231.1,...,5.0,278.0,8.0,279.0,2.0,279.1,15.0,735.0,183112800.0,7305.0
2020-09-23 14:22:29,250.0,260.0,234.5,248.2,5.0,242.2,6.0,242.0,10.0,231.1,...,5.0,278.0,8.0,279.0,2.0,279.1,15.0,735.0,183112800.0,7305.0
2020-09-23 14:22:30,250.0,260.0,234.5,248.2,5.0,242.2,6.0,242.0,10.0,231.1,...,5.0,278.0,8.0,279.0,2.0,279.1,15.0,735.0,183112800.0,7305.0


## Model Definition: LSTM Cell

In [5]:
class CustomLSTMCell(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(CustomLSTMCell, self).__init__()
        self.hidden_dim = hidden_dim

        # Initialize weights and biases
        self.W_i = nn.Parameter(torch.randn(input_dim, hidden_dim).float())
        self.U_i = nn.Parameter(torch.randn(hidden_dim, hidden_dim).float())
        self.b_i = nn.Parameter(torch.zeros(hidden_dim).float())

        self.W_f = nn.Parameter(torch.randn(input_dim, hidden_dim).float())
        self.U_f = nn.Parameter(torch.randn(hidden_dim, hidden_dim).float())
        self.b_f = nn.Parameter(torch.zeros(hidden_dim).float())
        
        self.W_c = nn.Parameter(torch.randn(input_dim, hidden_dim).float())
        self.U_c = nn.Parameter(torch.randn(hidden_dim, hidden_dim).float())
        self.b_c = nn.Parameter(torch.zeros(hidden_dim).float())
        
        self.W_o = nn.Parameter(torch.randn(input_dim, hidden_dim).float())
        self.U_o = nn.Parameter(torch.randn(hidden_dim, hidden_dim).float())
        self.b_o = nn.Parameter(torch.zeros(hidden_dim).float())
        
        # Layer normalization layers
        self.ln_i = nn.LayerNorm(hidden_dim)
        self.ln_f = nn.LayerNorm(hidden_dim)
        self.ln_c = nn.LayerNorm(hidden_dim)
        self.ln_o = nn.LayerNorm(hidden_dim)
        
        self.init_weights()

    def init_weights(self):
        for name, param in self.named_parameters():
            if 'W_' in name or 'U_' in name:
                nn.init.orthogonal_(param)  # Use orthogonal initialization
            elif 'b_' in name:
                nn.init.constant_(param, 0.0)

            # Check for NaN values
            assert(not torch.isnan(param).any())

    def forward(self, x, h, c):
        i_t = torch.sigmoid(self.ln_i(torch.mm(x, self.W_i) + torch.mm(h, self.U_i) + self.b_i))
        f_t = torch.sigmoid(self.ln_f(torch.mm(x, self.W_f) + torch.mm(h, self.U_f) + self.b_f))
        g_t = torch.tanh(self.ln_c(torch.mm(x, self.W_c) + torch.mm(h, self.U_c) + self.b_c))
        o_t = torch.sigmoid(self.ln_o(torch.mm(x, self.W_o) + torch.mm(h, self.U_o) + self.b_o))

        c_t = f_t * c + i_t * g_t
        h_t = o_t * torch.tanh(c_t)
        
        assert(not torch.isnan(h_t).any())
        assert(not torch.isnan(c_t).any())
    
        return h_t, c_t

## Model Definition: GRU Cell

In [6]:
class CustomGRUCell(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(CustomGRUCell, self).__init__()
        self.hidden_dim = hidden_dim

        # Initialize weights and biases
        self.W_z = nn.Parameter(torch.randn(input_dim, hidden_dim).float())
        self.U_z = nn.Parameter(torch.randn(hidden_dim, hidden_dim).float())
        self.b_z = nn.Parameter(torch.zeros(hidden_dim).float())

        self.W_r = nn.Parameter(torch.randn(input_dim, hidden_dim).float())
        self.U_r = nn.Parameter(torch.randn(hidden_dim, hidden_dim).float())
        self.b_r = nn.Parameter(torch.zeros(hidden_dim).float())
        
        self.W_h = nn.Parameter(torch.randn(input_dim, hidden_dim).float())
        self.U_h = nn.Parameter(torch.randn(hidden_dim, hidden_dim).float())
        self.b_h = nn.Parameter(torch.zeros(hidden_dim).float())
        
        # Layer normalization layers
        self.ln_z = nn.LayerNorm(hidden_dim)
        self.ln_r = nn.LayerNorm(hidden_dim)
        self.ln_h = nn.LayerNorm(hidden_dim)
        
        self.init_weights()

    def init_weights(self):
        for name, param in self.named_parameters():
            if 'W_' in name or 'U_' in name:
                nn.init.xavier_uniform_(param)  # Use orthogonal initialization
            elif 'b_' in name:
                nn.init.constant_(param, 0.0)

            # Check for NaN values
            assert(not torch.isnan(param).any())

    def forward(self, x, h):
        z_t = torch.sigmoid(self.ln_z(torch.mm(x, self.W_z) + torch.mm(h, self.U_z) + self.b_z))
        r_t = torch.sigmoid(self.ln_r(torch.mm(x, self.W_r) + torch.mm(h, self.U_r) + self.b_r))
        h_hat_t = torch.tanh(self.ln_h(torch.mm(x, self.W_h) + torch.mm(r_t * h, self.U_h) + self.b_h))

        h_t = (1 - z_t) * h + z_t * h_hat_t
        
        assert(not torch.isnan(h_t).any())
    
        return h_t

## Model Definition: RNN

In [7]:
class CustomRNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, dropout_rate, dense_units, cell_type='lstm'):
        super(CustomRNNModel, self).__init__()
        self.num_layers = num_layers
        self.hidden_dim = hidden_dim
        self.cell_type = cell_type

        if cell_type == 'lstm':
            self.layers = nn.ModuleList([CustomLSTMCell(input_dim, hidden_dim)])
            self.layers.extend([CustomLSTMCell(hidden_dim, hidden_dim) for _ in range(num_layers - 1)])
        elif cell_type == 'gru':
            self.layers = nn.ModuleList([CustomGRUCell(input_dim, hidden_dim)])
            self.layers.extend([CustomGRUCell(hidden_dim, hidden_dim) for _ in range(num_layers - 1)])
        else:
            raise ValueError("Unsupported cell type")

        self.dropout = nn.Dropout(dropout_rate)
        self.fc1 = nn.Linear(hidden_dim, dense_units)
        self.fc2 = nn.Linear(dense_units, output_dim)

    def forward(self, x):
        batch_size, seq_len, _ = x.size()

        h_t = [torch.zeros(batch_size, self.hidden_dim, device=x.device) for _ in range(self.num_layers)]
        c_t = [torch.zeros(batch_size, self.hidden_dim, device=x.device) for _ in range(self.num_layers)]

        for t in range(seq_len):
            x_t = x[:, t, :]
            for layer in range(self.num_layers):
                if self.cell_type == 'lstm':
                    h_t[layer], c_t[layer] = self.layers[layer](x_t, h_t[layer], c_t[layer])
                else:
                    h_t[layer] = self.layers[layer](x_t, h_t[layer])
                x_t = h_t[layer]

        x = self.dropout(x_t)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)

        return x

## RNN Model Training + Evaluation

In [8]:
# Loss definitions 
def quantile_loss(outputs, targets, quantile=0.5):
    errors = targets - outputs
    loss = torch.max((quantile - 1) * errors, quantile * errors)
    return torch.mean(loss)

class HingeLoss(nn.Module):
    def __init__(self):
        super(HingeLoss, self).__init__()
        
    def forward(self, outputs, targets):
        return torch.mean(torch.clamp(1 - targets * outputs, min=0))

def directional_loss(outputs, targets):
    return torch.mean(torch.abs(torch.sign(outputs) - torch.sign(targets)))

def choose_loss_function(loss):
    if loss == 'huber':  
        criterion = nn.SmoothL1Loss() 
    elif loss == 'mse': 
        criterion = nn.MSELoss()
    elif loss == 'quantile':
        criterion = quantile_loss(quantile=quantile)
    elif loss == 'hinge':
        criterion = HingeLoss()
    elif loss == 'directional':
        criterion = directional_loss()
    else:
        raise ValueError("Unsupported loss function")

    return criterion

class TrainAndPredict():
    def __init__(self, data_preparation, cell_type, train_needed=None):
        self.train_needed = train_needed
        self.test_loss, self.classification_accuracy, self.duration = self.train_and_predict(data_preparation, cell_type)

    # Extract hyperparameters
    def extract_hyperparams(self, dp, cell_type):
        num_units = dp.hyperparams[cell_type][cell_type + '_units']
        num_layers = dp.hyperparams[cell_type][cell_type + '_layers']
        dropout_rate = dp.hyperparams[cell_type]['dropout_rate']
        dense_units = dp.hyperparams[cell_type]['dense_units']
        init_learning_rate = dp.hyperparams[cell_type]['init_learning_rate']
        learning_rate_decay = dp.hyperparams[cell_type]['learning_rate_decay']
        init_epochs = dp.hyperparams[cell_type]['init_epochs']
        max_epochs = dp.hyperparams[cell_type]['max_epochs']
        early_stop_patience = dp.hyperparams[cell_type].get('early_stop_patience', None)
        
        if self.train_needed is None:
            self.train_needed = dp.hyperparams[cell_type]['pretrain'] # Whether to train the model
        
        return num_units, num_layers, dropout_rate, dense_units, init_learning_rate, learning_rate_decay, init_epochs, max_epochs, early_stop_patience, self.train_needed  


    def train_model(self, model, train_loader, val_loader, criterion, init_epochs, num_epochs, init_learning_rate, learning_rate_decay, device, early_stop_patience=None, cell_type='lstm'):
        train_losses = []
        val_losses = []
        best_val_loss = float('inf')
        early_stop_counter = 0
    
        # Initialize optimizer and scheduler
        optimizer = optim.Adam(model.parameters(), lr=init_learning_rate)
        scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs, eta_min=0.0)
    
        # Initialize gradient scaler for mixed precision training
        scaler = torch.cuda.amp.GradScaler()
    
        for epoch in range(num_epochs):
            model.train()
            current_lr = init_learning_rate * (learning_rate_decay ** max(float(epoch + 1 - init_epochs), 0.0))
            for param_group in optimizer.param_groups:
                param_group['lr'] = current_lr
    
            train_loss = 0.0
            with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA], profile_memory=True) as prof:
                for X_batch, y_batch in tqdm(train_loader):
                    X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                    with torch.cuda.amp.autocast():
                        outputs = model(X_batch)
                        loss = criterion(outputs, y_batch)
                    scaler.scale(loss).backward()
                    scaler.step(optimizer)
                    scaler.update()
                    optimizer.zero_grad()
                    train_loss += loss.item()
    
            train_loss /= len(train_loader)
            train_losses.append(train_loss)
    
            model.eval()
            val_loss = 0.0
            with torch.no_grad():
                for X_batch, y_batch in val_loader:
                    X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                    outputs = model(X_batch)
                    loss = criterion(outputs, y_batch)
                    val_loss += loss.item()
    
            val_loss /= len(val_loader)
            val_losses.append(val_loss)
    
            print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')
    
            # Update the learning rate
            scheduler.step()
    
            # Print profiling data
            print(prof.key_averages().table(sort_by="self_cuda_time_total", row_limit=10))
    
            # Early stopping
            if early_stop_patience is not None:
                if val_loss < best_val_loss:
                    best_val_loss = val_loss
                    early_stop_counter = 0
                    # Save best model
                    torch.save(model.state_dict(), f'models/SP500_{cell_type}_model.pth')
                else:
                    early_stop_counter += 1
                    if early_stop_counter >= early_stop_patience:
                        print("Early stopping triggered")
                        break
    
        return train_losses, val_losses

    
        
    # Training loop
    def train_model(self, model, train_loader, val_loader, criterion, init_epochs, num_epochs, init_learning_rate, learning_rate_decay, device, early_stop_patience=None, cell_type='lstm'):
        train_losses = []
        val_losses = []
        best_val_loss = float('inf')
        early_stop_counter = 0
    
        learning_rates_to_use = [
        init_learning_rate * (
            learning_rate_decay ** max(float(i + 1 - init_epochs), 0.0)
        ) for i in range(num_epochs)]
        
        for epoch in range(num_epochs):
            model.train()
            current_lr = learning_rates_to_use[epoch]
            optimizer = optim.Adam(model.parameters(), lr=current_lr)
            train_loss = 0.0
            for X_batch, y_batch in tqdm(train_loader):
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                optimizer.zero_grad()
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)
                loss.backward()
                optimizer.step()
                train_loss += loss.item()
    
            train_loss /= len(train_loader)
            train_losses.append(train_loss)
    
            model.eval()
            val_loss = 0.0
            with torch.no_grad():
                for X_batch, y_batch in val_loader:
                    X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                    outputs = model(X_batch)
                    loss = criterion(outputs, y_batch)
                    val_loss += loss.item()
    
            val_loss /= len(val_loader)
            val_losses.append(val_loss)
    
            print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')
    
            # Early stopping
            if early_stop_patience is not None:
                if val_loss < best_val_loss:
                    best_val_loss = val_loss
                    early_stop_counter = 0
                    # Save best model
                    torch.save(model.state_dict(), f'models/SP500_{cell_type}_model.pth')
                else:
                    early_stop_counter += 1
                    if early_stop_counter >= early_stop_patience:
                        print("Early stopping triggered")
                        break
        
        return train_losses, val_losses
    
    # Function to load the model
    def load_model(self, model, cell_type, device):
        model_path = f'models/SP500_{cell_type}_model.pth'
        model.load_state_dict(torch.load(model_path, map_location=device))
        model.to(device)
        return model

    # Function to make predictions
    def make_predictions(self, model, data_loader, device):
        model.eval()
        predictions = []
        actuals = []
        test_loss = 0
        with torch.no_grad():
            for X_batch, y_batch in data_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                outputs = model(X_batch)
                predictions.extend(outputs.cpu().numpy())
                actuals.extend(y_batch.cpu().numpy())
                criterion = choose_loss_function('huber')
                test_loss += criterion(outputs, y_batch)
        return np.array(predictions).flatten(), np.array(actuals).flatten(), test_loss.item()
    
    # Plot the predictions against the actual values
    def plot_results(self, data_preparation, actuals, predictions):
        plt.figure(figsize=(10, 5))
        x = np.arange(len(actuals))
        plt.bar(x - 0.2, actuals, label='Actual Prices')
        plt.bar(x + 0.2, predictions, label='Predicted Prices')
        plt.xlabel('Value')
        plt.ylabel('Normalized Price')
        plt.ylim(-0.1, 0.1)
        plt.title(f'Predicted vs Actual Prices for input_size={data_preparation.input_size}, num_steps={data_preparation.num_steps}')
        plt.legend()
        plt.show()
    
    # Check for overfitting/underfitting
    def plot_losses(self, train_losses, val_losses):
        plt.figure(figsize=(10, 5))
        plt.plot(train_losses, label='Train Loss')
        plt.plot(val_losses, label='Validation Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        plt.show()
    
    def train_and_predict(self, data_preparation, cell_type):
        # Initialize model
        num_units, num_layers, dropout_rate, dense_units, init_learning_rate, learning_rate_decay, init_epochs, max_epochs, early_stop_patience, train_needed = self.extract_hyperparams(data_preparation, cell_type)
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        model = CustomRNNModel(input_dim=data_preparation.input_size, hidden_dim=num_units, output_dim=data_preparation.input_size, num_layers=num_layers, dropout_rate=dropout_rate, dense_units=dense_units, cell_type=cell_type).to(device)
        
        # Train or load the model
        if train_needed:
            # Train the model
            criterion = choose_loss_function('huber')
            t1 = time.time()
            train_losses, val_losses = self.train_model(model, data_preparation.train_loader, data_preparation.val_loader, criterion, init_epochs, max_epochs, init_learning_rate, learning_rate_decay, device, early_stop_patience, cell_type=cell_type)
            duration = time.time() - t1
            
            torch.save(model.state_dict(), f'models/SP500_{cell_type}_model.pth')
        else:
            # Load the model
            model = self.load_model(model, cell_type, device)
            print("Model loaded successfully")
        
        # Get a model summary
        print(model)
        
        # Get predictions and actual values
        predictions, actuals, test_loss = self.make_predictions(model, data_preparation.test_loader, device)
        
        # Plot the predictions against the actual values
        self.plot_results(data_preparation, actuals, predictions)
        
        # Check for overfitting/underfitting
        self.plot_losses(train_losses, val_losses)
        
        # Classification rate
        classification_accuracy = np.mean((predictions > 0) == (actuals > 0))
    
        return test_loss, classification_accuracy, duration

In [9]:
dataset_name = 'INE.sc2010'
period = '1S'

In [ ]:
data_preparation = DataPreparation(dataset_name, period=period)  # Adjust num_steps and input_size as needed
cell_type = 'lstm'
evaluation_results = TrainAndPredict(data_preparation, cell_type, train_needed=True)

 85%|███████████████████████████████▍     | 81031/95348 [16:42<02:46, 86.05it/s]

In [ ]:
data_preparation = DataPreparation(dataset_name, period=period)
cell_type = 'gru'
evaluation_results = TrainAndPredict(data_preparation, cell_type, train_needed=True))

## Hyperparameter Search over `input_size` and `num_steps`

In [ ]:
def grid_search(cell_type, input_size_values, num_steps_values):
    # Define dataframe
    df = pd.DataFrame(columns=['Input Size', 'Number of Steps', 'Test Loss', 'Classification Accuracy', 'Duration'])
    
    # Define test losses & classification accuracies
    input_size_combination_values = []
    num_steps_combination_values = []

    # Define hyperparam combinations
    test_losses = []
    classification_accuracies = []
    durations = []
    
    # Loop over the hyperparam values
    for input_size in input_size_values:
        for num_steps in num_steps_values:
            # Add to hyperparam combination
            input_size_combination_values.append(input_size)
            num_steps_combination_values.append(num_steps)
            
            # Modify the input_size and num_steps attributes
            data_preparation = DataPreparation(dataset_name, num_steps=num_steps, input_size=input_size)
            
            # Train & Evaluate model
            evaluation_results = TrainAndPredict(data_preparation, cell_type, train_needed=True)
            test_losses.append(evaluation_results.test_loss)
            classification_accuracies.append(evaluation_results.classification_accuracy)   
            durations.append(evaluation_results.duration)
    
    df['Input Size'] = input_size_combination_values
    df['Number of Steps'] = num_steps_combination_values
    df['Test Loss'] = test_losses
    df['Classification Accuracy'] = classification_accuracies
    df['Duration'] = durations
    
    df.to_csv(f'results/{cell_type}_evaluation_results.csv')

In [ ]:
def get_best_model(cell_type, input_size_values, num_steps_values, SEARCH_NEEDED=False):
    # Perform grid search if needed (setting SEARCH_NEEDED = TRUE will RUN LONG TIME!!!)
    if SEARCH_NEEDED:
        grid_search(cell_type, input_size_values, num_steps_values)
    
    # Get best model
    df = pd.read_csv(f'results/{cell_type}_evaluation_results.csv')
    idx = np.argmin(df['Test Loss'])
    df_best_model = df.iloc[idx, :]
    return df_best_model

In [ ]:
input_size_values = [2, 3, 5, 10, 20]
num_steps_values = [3, 10, 20, 30, 40, 50]

In [ ]:
# LSTM (setting SEARCH_NEEDED = TRUE will RUN LONG TIME!!!)
cell_type = 'lstm'
SEARCH_NEEDED_LSTM = False
df_best_model = get_best_model(cell_type, input_size_values, num_steps_values, SEARCH_NEEDED=SEARCH_NEEDED_LSTM)

In [ ]:
df_best_model

In [ ]:
# Modify the input_size and num_steps attributes
data_preparation = DataPreparation(dataset_name, num_steps=int(df_best_model['Number of Steps']), input_size=int(df_best_model['Input Size']))

# Train & Evaluate model
evaluation_results = TrainAndPredict(data_preparation, cell_type, train_needed=True)
print("Test loss of best model:", evaluation_results.test_loss)
print("Classification accuracy of best model:", evaluation_results.classification_accuracy)
print("Duration of best model:", evaluation_results.duration)

In [ ]:
# GRU (setting SEARCH_NEEDED = TRUE will RUN LONG TIME!!!)
cell_type = 'gru'
SEARCH_NEEDED_GRU = True
df_best_model = get_best_model(cell_type, input_size_values, num_steps_values, SEARCH_NEEDED=SEARCH_NEEDED_GRU)

In [ ]:
# import numpy as np
# from scipy.special import kl_div

# # Sample data for demonstration purposes
# # In practice, you would use actual activation data from the model
# def get_fp32_activation_data():
#     # Replace this with actual data collection
#     return np.array([0.5, 1.0, 2.0, -1.0, -0.5, 3.0, -3.0, 0.1])

# # **1. Calibration: Collect Data and Compute Scale Factor**

# # Collect activation data (replace with actual FP32 data)
# activation_data_fp32 = get_fp32_activation_data()

# # Step 1: Determine the maximum absolute value
# max_abs_value = np.max(np.abs(activation_data_fp32))

# # Step 2: Compute the scale factor for symmetric quantization
# def compute_scale(max_abs_value):
#     return max_abs_value / 127  # 127 because 8-bit quantization uses values from -128 to 127

# scale = compute_scale(max_abs_value)

# # Step 3: Perform preliminary quantization of FP32 data
# def preliminary_quantize(fp32_data, scale):
#     return np.clip(np.round(fp32_data / scale), -128, 127).astype(np.int8)

# activation_data_int8 = preliminary_quantize(activation_data_fp32, scale)

# # Step 4: Compute histograms for FP32 and INT8
# def compute_histograms(fp32_data, int8_data):
#     hist_fp32, _ = np.histogram(fp32_data, bins=2048, range=(-128, 127), density=True)
#     hist_int8, _ = np.histogram(int8_data, bins=2048, range=(-128, 127), density=True)
#     return hist_fp32, hist_int8

# hist_fp32, hist_int8 = compute_histograms(activation_data_fp32, activation_data_int8)

# # Step 5: Compute KL divergence
# def compute_kl_divergence(hist_fp32, hist_int8):
#     # Adding a small constant to avoid log(0)
#     return np.sum(kl_div(hist_fp32 + 1e-8, hist_int8 + 1e-8))

# kl_divergence = compute_kl_divergence(hist_fp32, hist_int8)
# print(f"KL Divergence: {kl_divergence}")

# # **2. Quantization: Convert FP32 to INT8**

# # Final quantization of FP32 data
# def quantize_to_int8(fp32_data, scale):
#     return np.clip(np.round(fp32_data / scale), -128, 127).astype(np.int8)

# activation_data_int8_final = quantize_to_int8(activation_data_fp32, scale)

# # **3. INT32 Computations: Perform Layer Operations**

# # Example INT32 computation function
# def int32_computations(weights, activations, bias):
#     # Perform INT32 matrix multiplication and add bias
#     int32_result = np.dot(weights, activations) + bias
#     return int32_result

# # Sample weights and bias for demonstration
# weights = np.array([[1, -1], [2, 3]])
# bias = np.array([1, -1])

# # Perform INT32 computations
# int32_result = int32_computations(weights, activation_data_int8_final, bias)
# print(f"INT32 Computation Result: {int32_result}")

# # **4. Re-Quantization: Convert INT32 to INT8**

# # Re-quantization process
# def requantize(int32_activations, scale, zero_point, bias):
#     # Add bias and then requantize
#     int32_activations_with_bias = int32_activations + bias
#     return np.clip(np.round(int32_activations_with_bias * scale) + zero_point, -128, 127).astype(np.int8)

# # Assuming zero_point = 0 for symmetric quantization
# zero_point = 0

# # Re-quantize INT32 results to INT8
# activation_data_int8_requantized = requantize(int32_result, scale, zero_point, bias)
# print(f"Re-Quantized INT8 Data: {activation_data_int8_requantized}")

# # **5. De-Quantization: Convert INT8 Back to FP32**

# # De-quantization process
# def dequantize_to_fp32(int8_data, scale, zero_point):
#     return (int8_data - zero_point) * scale

# # Convert INT8 results back to FP32
# fp32_reconstructed_data = dequantize_to_fp32(activation_data_int8_requantized, scale, zero_point)
# print(f"De-Quantized FP32 Data: {fp32_reconstructed_data}")